In [ ]:
import pandas as pd
import re

# =============================================================================
# Section 2: Data Loading, Cleaning, and Merging
# =============================================================================
print("\n--- Section 2: Loading, Cleaning & Merging Data ---")

def clean_and_prepare_mepfs_data(df, file_type='quantity'):
    """A reusable function to perform initial cleaning on the MEPFS dataframes."""
    # Rename the first column to 'Project_Name'
    df.rename(columns={df.columns[0]: 'Project_Name'}, inplace=True)
    
    # Drop irrelevant columns and clean project names
    df.drop(columns=['MEPFS aspect'], inplace=True, errors='ignore')
    df['Project_Name'] = df['Project_Name'].str.strip().str.lower()

    # ✅ Apply the "1x5 → 1 STY 5 CLS" transformation
    df['Project_Name'] = df['Project_Name'].apply(transform_project_name)
    
    # Define the specific feature columns for the MEPFS dataset
    mepfs_feature_cols = [
        'Fire alarm system', 'Panelboard', 'Lighting fixtures', 'Conduits',
        'Sewer Line works', 'Plumbing fixtures', 'Wires'
    ]

    # Clean numeric columns: remove commas, handle non-numeric placeholders
    for col in mepfs_feature_cols:
        if col in df.columns:
            df[col] = df[col].astype(str).str.replace(',', '', regex=False).str.replace('-', 'NaN', regex=False)
            df[col] = pd.to_numeric(df[col], errors='coerce')

    # Impute missing numeric values with the column median
    for col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            if df[col].isnull().any():
                median_val = df[col].median()
                df[col].fillna(median_val, inplace=True)
    
    return df

def transform_project_name(text):
    """
    Converts patterns like '1x5' into '1 STY 5 CLS'.
    Works dynamically for any 'NxM' pattern.
    """
    if not isinstance(text, str):
        return text
    
    match = re.search(r'(\d+)\s*[xX]\s*(\d+)', text)
    if match:
        n, m = match.groups()
        return text.replace(match.group(0), f"{n} STY {m} CLS")
    return text

def extract_year_budget(text):
    """Extracts year and budget from the 'Year/Budget' string."""
    if not isinstance(text, str):
        return None, None
    
    # Extract year (a 4-digit number)
    year_match = re.search(r'\b(20\d{2})\b', text)
    year = int(year_match.group(1)) if year_match else None
    
    # Extract budget (a number that can have commas)
    budget_match = re.search(r'[:\s]([\d,]+\.?\d*)', text)
    budget = float(budget_match.group(1).replace(',', '')) if budget_match else None
    
    return year, budget

try:
    df_quantity = pd.read_csv('MEPFS Quantity Cost.csv')
    print("MEPFS Quantity data loaded.")
    df_unit_cost = pd.read_csv('MEPFS Unit Cost.csv')
    print("MEPFS Unit Cost data loaded.")
except FileNotFoundError as e:
    print(f"Error: {e}. Make sure both CSV files are in the correct directory.")
    exit()

# --- Clean and Merge Dataframes ---
df_quantity_cleaned = clean_and_prepare_mepfs_data(df_quantity.copy())
df_unit_cost_cleaned = clean_and_prepare_mepfs_data(df_unit_cost.copy(), file_type='unit_cost')

# Extract Year and Budget
df_quantity_cleaned[['Year', 'Budget']] = df_quantity_cleaned['Year/Budget'].apply(extract_year_budget).apply(pd.Series)
df_quantity_cleaned.drop(columns=['Year/Budget'], inplace=True)

# Merge the two dataframes
df_merged = pd.merge(
    df_quantity_cleaned,
    df_unit_cost_cleaned,
    on='Project_Name',
    suffixes=('_qty', '_cost')
)

# Drop rows where budget is missing and filter out unusually low budgets
df_merged = df_merged.dropna(subset=['Budget'])
df_merged = df_merged[df_merged['Budget'] > 100000].copy()
print(f"Dataframes merged. Working with {len(df_merged)} common projects.")
